In [2]:
import numpy as np
# np.set_printoptions(precision=3)

def generate_non_singular_matrix(n):
    while True:
        A = np.random.rand(n, n) + 1j * np.random.rand(n, n)
        if np.linalg.det(A) != 0:
            return A

In [3]:
def QR_decomposition(A):
    n = A.shape[0]
    Q = np.zeros((n, n))
    R = np.zeros((n, n))

    for k in range(n):
        Q[:, k] = A[:, k]
        for i in range(k):
            R[i, k] = np.dot(Q[:, i], A[:, k])
            Q[:, k] -= R[i, k] * Q[:, i]
        Q[:, k] /= np.linalg.norm(Q[:, k])
        R[k, k] = np.dot(np.conj(Q[:, k]), A[:, k])

    return Q, R

In [4]:
def permutation(v):
    """
    Constructs a permutation matrix from a complex vector following the order of the modulus of the entries.
    Parameters:
        v (ndarray): Complex vector.
    Returns:
        P (ndarray): Permutation matrix.
    """
    # Sort indices of v based on the modulus of entries
    sorted_indices = np.argsort(abs(v))[::-1]

    # Create permutation matrix
    n = len(v)
    P = np.zeros((n, n))
    for i, idx in enumerate(sorted_indices):
      P[i, idx] = 1

    return P

In [5]:
def qr_method(A, tol, x):
    """
    QR method to compute eigenvalues.

    Parameters:
        A (ndarray): Matrix A.
        tol (float): Tolerance for loop exit (default=1e-6).
        x (ndarray): Initial guess (optional).
    Returns:
        Ak (ndarray): Upper triangular matrix with diagonal values sorted by their norm.
        V (ndarray): Invertible matrix such that A = V Ak V^*.
    """
    error = tol + 1
    Ak = np.copy(A)
    V = np.eye(A.shape[0])

    while error > tol:
        Atmp = np.copy(Ak)

        epsilon = np.random.randn()
        Qk, Rk = QR_decomposition(Ak + epsilon * np.eye(Ak.shape[0]))
        Ak = Rk @ Qk - epsilon * np.eye(Ak.shape[0])

        P = permutation(np.diag(Ak))
        Ak = np.linalg.inv(P) @ Ak @ P
        V = V @ Qk @ P

        error = np.linalg.norm(np.diag(Ak) - np.diag(Atmp))

    return Ak, V

In [6]:
tol=1e-20

In [7]:
# Test on a random 4x4 Hermitian matrix A
np.random.seed(42)
# A = np.random.rand(4, 4) + 1j * np.random.rand(4, 4)
A = generate_non_singular_matrix(4)
A = (A + A.conj().T) / 2  # Making A Hermitian
print("Random Hermitian matrix A:")
print(A)

Random Hermitian matrix A:
[[0.375+0.j    0.553-0.044j 0.667-0.012j 0.716-0.151j]
 [0.553+0.044j 0.156+0.j    0.383-0.247j 0.539+0.16j ]
 [0.667+0.012j 0.383+0.247j 0.021+0.j    0.576-0.047j]
 [0.716+0.151j 0.539-0.16j  0.576+0.047j 0.183+0.j   ]]


In [8]:
x = np.random.rand(A.shape[0])
print("Initial guess x: ", x)

Initial guess x:  [0.065 0.949 0.966 0.808]


In [9]:
# Compute Ak and V using QR method
Ak, V = qr_method(A, tol = tol, x = x)
print("\nUpper triangular matrix Ak:")
Aktmp = np.copy(Ak)
for i in range(4):
  for j in range(4):
    if abs(Aktmp[i,j]) < 1e-6:
      Aktmp[i,j] = 0
print(Aktmp)
print("\nInvertible matrix V:")
print(V)

<ipython-input-3-526e4577d786>:7: ComplexWarning: Casting complex values to real discards the imaginary part
  Q[:, k] = A[:, k]
<ipython-input-3-526e4577d786>:12: ComplexWarning: Casting complex values to real discards the imaginary part
  R[k, k] = np.dot(np.conj(Q[:, k]), A[:, k])
<ipython-input-3-526e4577d786>:9: ComplexWarning: Casting complex values to real discards the imaginary part
  R[i, k] = np.dot(Q[:, i], A[:, k])



Upper triangular matrix Ak:
[[ 1.934  0.     0.     0.   ]
 [ 0.    -0.51   0.     0.   ]
 [ 0.     0.    -0.45   0.   ]
 [ 0.     0.     0.    -0.24 ]]

Invertible matrix V:
[[-0.584  0.389  0.638  0.317]
 [-0.435 -0.204  0.155 -0.863]
 [-0.447 -0.795 -0.119  0.392]
 [-0.52   0.418 -0.745  0.03 ]]


 (ii) $VA_kV^∗ = A$

In [10]:
# Check if A = V Ak V^*
reconstructed_A = V @ Ak @ np.linalg.inv(V)
print("\nReconstructed matrix:")
print(reconstructed_A)

print("\nReconstructed matrixis equals to A: ", tol >= np.linalg.norm(A-reconstructed_A))


Reconstructed matrix:
[[0.375 0.553 0.667 0.716]
 [0.553 0.156 0.383 0.539]
 [0.667 0.383 0.021 0.576]
 [0.716 0.539 0.576 0.183]]

Reconstructed matrixis equals to A:  False


(iii) the eigenvector associated to the biggest eigenvalue of $A$ is the first column of the output matrix $V$.

In [11]:
# Check if eigenvector associated with the biggest eigenvalue of A is the first column of V
eigenvals, eigenvectors = np.linalg.eigh(A)

biggest_eigvec = eigenvectors[:, -1]
first_col_V = V[:, 0]
print("\nEigenvector associated with the biggest eigenvalue of A:")
print(biggest_eigvec)
print("\nFirst column of V:")
print(first_col_V)


Eigenvector associated with the biggest eigenvalue of A:
[-0.583+0.j    -0.436-0.031j -0.441-0.069j -0.516-0.054j]

First column of V:
[-0.584 -0.435 -0.447 -0.52 ]
